In [48]:
from denoise import AudioDeNoise 

audioDenoiser = AudioDeNoise(inputFile="/home/aicontest/DF/test/moon/workspace/TEST_00229.ogg")
audioDenoiser.deNoise(outputFile="input_denoised.wav")
# audioDenoiser.generateNoiseProfile(noiseFile="input_noise_profile.wav")

LibsndfileError: Error opening '/home/aicontest/DF/test/moon/workspace/TEST_00229.ogg': System error.

In [5]:
from transformers import AutoFeatureExtractor, ASTForAudioClassification
from datasets import load_dataset
import torch
import torchaudio

data, sr = torchaudio.load('/home/aicontest/DF/data/audio/test/TEST_47953.ogg')
data = torchaudio.functional.resample(data, orig_freq=sr, new_freq=16000)
data = data.squeeze()

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation", trust_remote_code=True)
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

# print(dataset[0]['audio']['array'])

feature_extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

# audio file is decoded on the fly
inputs = feature_extractor(data, sampling_rate=16000, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

logits = logits.squeeze()


predicted_class_ids = torch.argsort(logits)[-5:]
# print(predicted_class_ids)

for _id in predicted_class_ids:
    predicted_label = model.config.id2label[_id.item()]
    print(predicted_label)


# compute loss - target_label is e.g. "down"
target_label = model.config.id2label[0]
inputs["labels"] = torch.tensor([model.config.label2id[target_label]])
loss = model(**inputs).loss
round(loss.item(), 2)

Musical instrument
Music
Door
Sliding door
Speech


0.5

In [30]:
model.config.id2label[136]

'Whale vocalization'

In [35]:

feature_extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

model = model.to('cuda:0')

def inference_top_k(fpath, k=5, vname='Sliding door'):
    data, sr = torchaudio.load(fpath)
    data = torchaudio.functional.resample(data, orig_freq=sr, new_freq=16000)
    data = data.squeeze()

    # audio file is decoded on the fly
    inputs = feature_extractor(data, sampling_rate=16000, return_tensors="pt")
    inputs = inputs.to('cuda:0')
    with torch.no_grad():
        logits = model(**inputs).logits

    logits = logits.squeeze()


    predicted_class_ids = torch.argsort(logits)[-k:]
    predicted_label = [model.config.id2label[_id.item()] for _id in predicted_class_ids]

    return predicted_label
    # # print(fpath, predicted_label)

    # if 'Speech' not in predicted_label:
    #     print(fpath, f"여기서 {vname} 소리가 남")

    # # if model.config.id2label[predicted_class_ids[-1].item()] == vname:
    # #     print(fpath)

In [38]:
from glob import glob
from tqdm import tqdm

tqdm_bar = tqdm(glob('/home/aicontest/DF/data/audio/test/*.ogg'))
for file in tqdm_bar:
    result = inference_top_k(file, 5, 'Ocean')

    tqdm_bar.set_postfix(Prediction=result)
    if 'Speech' not in result:
        print(file)


 23%|██▎       | 288/1264 [00:07<00:26, 37.21it/s, Prediction=['Animal', 'Domestic animals, pets', 'Inside, small room', 'Throat clearing', 'Speech']]                               

/home/aicontest/DF/data/audio/unlabeled_data/XJQCCIWQ.ogg
/home/aicontest/DF/data/audio/unlabeled_data/AJEUSDFU.ogg
/home/aicontest/DF/data/audio/unlabeled_data/BZXAPCTJ.ogg


 28%|██▊       | 356/1264 [00:09<00:23, 37.91it/s, Prediction=['Silence', 'Narration, monologue', 'Male speech, man speaking', 'Speech synthesizer', 'Speech']]               

/home/aicontest/DF/data/audio/unlabeled_data/JJEZFIWP.ogg


 31%|███       | 388/1264 [00:10<00:23, 37.58it/s, Prediction=['Music', 'Rain on surface', 'White noise', 'Vehicle', 'Speech']]                                          

...

 97%|█████████▋| 1220/1264 [00:33<00:01, 34.23it/s, Prediction=['Insect', 'Speech', 'Rain', 'Raindrop', 'Rain on surface']]                                           

/home/aicontest/DF/data/audio/unlabeled_data/IMHXBPNV.ogg
/home/aicontest/DF/data/audio/unlabeled_data/LPAOYJPR.ogg
/home/aicontest/DF/data/audio/unlabeled_data/ZBZNGKVY.ogg


 99%|█████████▊| 1248/1264 [00:34<00:00, 36.80it/s, Prediction=['Frog', 'Raindrop', 'Rain on surface', 'Rain', 'Speech']]                                               

/home/aicontest/DF/data/audio/unlabeled_data/GJNSVOZO.ogg
/home/aicontest/DF/data/audio/unlabeled_data/DTAKNIPW.ogg


100%|██████████| 1264/1264 [00:34<00:00, 36.78it/s, Prediction=['Vehicle', 'Rustling leaves', 'Pink noise', 'White noise', 'Speech']]               


In [48]:
import torch
import torchaudio
from speechbrain.inference.enhancement import SpectralMaskEnhancement

enhance_model = SpectralMaskEnhancement.from_hparams(
    source="speechbrain/metricgan-plus-voicebank",
    savedir="pretrained_models/metricgan-plus-voicebank",
    run_opts={"device":"cuda"}
)

def moons_removal(fname):
    # Load and add fake batch dimension
    noisy = enhance_model.load_audio(fname).unsqueeze(0)

    # Add relative length tensor
    enhanced = enhance_model.enhance_batch(noisy, lengths=torch.tensor([1.]))

    return enhanced.cpu()
    # # Saving enhanced signal on disk
    # torchaudio.save('enhanced.wav', enhanced.cpu(), 16000)


In [50]:
from IPython.display import Audio
audio_path = "/home/aicontest/DF/data/audio/unlabeled_data/ACKIWIQZ.ogg"
y, sr = torchaudio.load(audio_path)

Audio(y.squeeze().numpy(), rate=sr)


In [51]:
removed = moons_removal(audio_path)
Audio(removed, rate=16000)

In [59]:
tqdm_bar = tqdm(glob('/home/aicontest/DF/data/audio/test/*.ogg'))
for file in tqdm_bar:
    removed = moons_removal(file)

    file_name = file.split("/")[-1][:-3] + "wav"
    file_path = f"/home/aicontest/DF/data/audio/moon_test_data/{file_name}"
    torchaudio.save(file_path, removed, 16000)

100%|██████████| 50000/50000 [12:11<00:00, 68.39it/s]  
